In [1]:
# from twitter_scraper import get_tweets
import os
import sys
sys.path.insert(0, os.path.abspath('../../Modules/twitter-scraper/twitter_scraper/modules'))

import pandas as pd
import spacy

from tweets import get_tweets
from functions import clean_description
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
users = ['DannyDeVito', 'hannibalburess']
tweets = []
tweet_user = []

for user in users:
    for tweet in get_tweets(user, pages=100):
        tweets.append(tweet['text'])
        tweet_user.append(user)

In [3]:
tweets[:5]

['Having private health insurance doesn’t mean a damn thing if you have a $5,000 deductible that you can’t afford. \n\nThat is why Medicare for All will eliminate deductibles, along with co-payments and premiums.',
 'Trollfoot wishes you kindness babies, kindness pic.twitter.com/NU3RMRJ52C',
 'WATCH as Rep. Raskin rips apart the Republican’s line that Donald J. Trump is an  “anti-corruption crusader. pic.twitter.com/O3iPYNqmrn',
 'The Gang Beat Boris https://twitter.com/UKLabour/status/1202972873712910342\xa0 pic.twitter.com/ovesBsu7Sq',
 'We can’t let him get away with this pic.twitter.com/iL5hdRZUUV']

## Clean & Tokenize

In [4]:
# Send to df
data = {'tweets':tweets, 'user':tweet_user}
df = pd.DataFrame(data)
df.head()

,tweets,user
0,Having private health insurance doesn’t mean a...,DannyDeVito
1,"Trollfoot wishes you kindness babies, kindness...",DannyDeVito
2,WATCH as Rep. Raskin rips apart the Republican...,DannyDeVito
3,The Gang Beat Boris https://twitter.com/UKLabo...,DannyDeVito
4,We can’t let him get away with this pic.twitte...,DannyDeVito


In [5]:
# NLP Model
nlp = spacy.load("en_core_web_md")

In [6]:
# send clean text to list
text = df['tweets'].apply(clean_description).tolist()

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://twitter.com/alwayssunny/status/1197209939573272576 …" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://twitter.com/jumanjimovie/status/1189889761449775105 …" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://www.nytimes.com/2019/03/16/opinion/sunday/chess-champion-8-year-old-homeless-refugee-.html?action=click&module=Opinion&pgtype=Homepage …" looks like a URL. Beaut

In [9]:
# Instantiate Vectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

In [10]:
def fit_for_nn(text_list):
    # Create a vocab and get word counts per doc
    sparse = tfidf.fit_transform(text_list)
    # send to df
    tfidf_dtm = pd.DataFrame(sparse.todense(), columns = tfidf.get_feature_names())
    return tfidf_dtm

tfidf_dtm = fit_for_nn(text)

In [11]:
tfidf_dtm.head()

,00xjr0lmbb,01,03,04,05,052316okay,06,07,08,080054cf4b86ac27,...,zorros,zpclhfxutj,zpea1mdaxwpictwittercom,zuul,zx0rptchou,zx6lhjpvaf,zxmeftyteo,zxxwkmthnq,zyy0nwgo0l,zzec7agwsreftw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model

In [12]:
# Instantiate model
nn = NearestNeighbors(n_neighbors=20, algorithm='ball_tree')
nn.fit(tfidf_dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [13]:
def transform_query_for_nn(string):
    # Create a vocab and get word counts per doc
    sparse = tfidf.transform([query])
    query_dtm = pd.DataFrame(sparse.todense(), columns = tfidf.get_feature_names())
    return query_dtm

In [24]:
query = """
dog"""

query_dtm = transform_query_for_nn(query)

In [25]:
# Query for closest neighbors
results = nn.kneighbors(query_dtm)[1][0].tolist()

In [26]:
# Send to list
closest_tweets = df.ix[results]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


In [27]:
closest_tweets.head()

,tweets,user
455,#Trollfoot dog M pic.twitter.com/zYY0nWgo0l,DannyDeVito
1537,,hannibalburess
246,Muhammad (sa): A man saw a thirsty dog & climb...,DannyDeVito
142,“Seeds.”\n\nOut of the minority of seats they ...,DannyDeVito
1215,Another one. March 15th @JoeBudden presents “...,hannibalburess


In [28]:
for index, row in closest_tweets.iterrows():
    print("")
    print("---")
    print("")
    print(f"{row['user']}: {row['tweets']}")


---

DannyDeVito: #Trollfoot dog M pic.twitter.com/zYY0nWgo0l

---

hannibalburess: 

---

DannyDeVito: Muhammad (sa): A man saw a thirsty dog & climbed into a well to give it water
Companions: O Prophet—is there a reward for charity even for the animals?
Muhammad (sa): In every living being there is a reward for charity
(Sahih Muslim 2244)
#LosAngelesFire pic.twitter.com/A7fdPrH5QE

---

DannyDeVito: “Seeds.”

Out of the minority of seats they have in Congress, the GOP allowed a white supremacist to represent ~700,000 people.

Republicans won’t even censure him.

They also protect a president who can’t bring himself to condemn neo-Nazis.

Those aren’t seeds.
They’re redwoods https://twitter.com/washingtonpost/status/1083964094171045888 …

---

hannibalburess: Another one. March 15th @JoeBudden presents  “Oh What a Night” at the @HLBallroom. With special guest @hannibalburess All submissions go to (OhWhataNightNYC@gmail.com). Come rock with us!

Check out the link below for more info 